
# Chapter 13 Liquidity
- Definition: A security’s liquidity refers to the ease with which the security can be bought and/or sold

### - Liquidity & expected stock returns
- Illiquidity is positively related to expected market returns in both the time series and cross section. 

### - Aggregate liquidity factor

### 13.1 Measuring Liquidity
- Roll(1984):
![Markdown](http://i1.fuimg.com/716668/3541ed60e17c71aa.png)
- Chordia(2001):  
   - Quoted spread:  difference between the bid and the ask price(in dollars / percentage of price)
   - Effective spread:   difference between the midpoint of the best bid and best offer prices and the execution price of a trade. (in dollars / percentage of price)
   - Market depth:  the average of the number of shares available at the best bid and the best offer
   - Dollar depth:   average of the best bid price times the number of shares available at that price and the best offer price times the number of shares available at the offer price
   - Volume, dollar volume, and the number of trades……
- Amihud(2002)(illiquidity)   
  - ![Markdown](http://i1.fuimg.com/716668/9f306a740b22ae8e.png)
  - The value of Illiq can be interpreted as the percentage price impact of trading one million dollars 
  - 4 measurement periods     
      - one (Illiq1M), three (Illiq3M), six (Illiq6M), and 12 months (Illiq12M) worth of data up to and including the month t 
      - highly skewed -> ln(1+illiq)
      - We require a minimum of 15, 50, 100, and 200 valid daily observations to calculate Illiq1M, Illiq3M, Illiq6M, and Illiq12M, respectively -> Missing Feb.

## 13.2 Summary Statistics

In [2]:
import os
import pandas as pd
import numpy as np
import pandas as pd

In [ ]:
a = pd.read_csv('liqdata.csv',index_col = 0)
a.columns = ['code','date','vold','ret']
a.date = pd.to_datetime(a.date)
a.vold = a.vold / 1000000
a.ret = a.ret * 100
a.ret = a.ret.abs()
a['rv'] = a.ret / a.vold

allstock = a.code.unique().tolist()
ts = pd.date_range(start='19980101',end='20200101',freq = 'm')

liq = pd.DataFrame(columns = ['code','1m','3m','6m','12m'])
for s in allstock:
    subdata = a[a.code == s]
    subliq = pd.DataFrame(index = ts,columns = ['code','1m','3m','6m','12m'])
    subliq.code = s
    for t in range(12,len(ts)):
        # 1m
        subdata1 = subdata[(subdata.date <= ts[t])&(subdata.date > ts[t-1])]
        if len(subdata1) < 15:
            subliq.iloc[t,1] = np.nan
        else:
            subliq.iloc[t,1] = subdata1.rv.mean()
        # 3m
        subdata3 = subdata[(subdata.date <= ts[t])&(subdata.date > ts[t-3])]
        if len(subdata3) < 50:
            subliq.iloc[t,2] = np.nan
        else:
            subliq.iloc[t,2] = subdata3.rv.mean()
        # 6m
        subdata6 = subdata[(subdata.date <= ts[t])&(subdata.date > ts[t-6])]
        if len(subdata6) < 100:
            subliq.iloc[t,3] = np.nan
        else:
            subliq.iloc[t,3] = subdata6.rv.mean()
        # 12m
        subdata12 = subdata[(subdata.date <= ts[t])&(subdata.date > ts[t-12])]
        if len(subdata12) < 200:
            subliq.iloc[t,4] = np.nan
        else:
            subliq.iloc[t,4] = subdata12.rv.mean()
    liq = liq.append(subliq.iloc[12:,:])
    print(s)
        
liq.columns = ['code','illiq1M','illiq3M','illiq6M','illiq12M']
liq['lnilliq1M'] = np.log(liq.illiq1M.tolist())
liq['lnilliq3M'] = np.log(liq.illiq3M.tolist())
liq['lnilliq6M'] = np.log(liq.illiq6M.tolist())
liq['lnilliq12M'] = np.log(liq.illiq12M.tolist())

liq.to_csv('liqresult.csv')

In [ ]:
liqdata.index = pd.to_datetime(liqdata.index)

liqdata1 = liqdata[liqdata.index > liqdata.index[11]]
table1 = pd.DataFrame(index = ['illiq1M','illiq3M','illiq6M','illiq12M',
                               'lnilliq1M','lnilliq3M','lnilliq6M','lnilliq12M'],
    columns = ['mean','SD','skew','kurt','min','5%','25%','median',
               '75%','95%','max','n'])
def month_sum(s):
#    s = s.dropna()
    me,SD,sk,ku = s.mean(),s.std(),s.skew(),s.kurt()
    mi,ma = s.min(),s.max()
    p5,p25,p50,p75,p95 = np.percentile(s,[5,25,50,75,95])
    n = len(s)
    return me,SD,sk,ku,mi,p5,p25,p50,p75,p95,ma,n
def mean_sum(var):
    vardata = liqdata1[var]
    ts = pd.DataFrame(index = vardata.index.unique(),columns = ['mean','SD','skew',
                      'kurt','min','5%','25%','median','75%','95%','max','n'])
    for t in ts.index:
        s = vardata[vardata.index == t]
        s = s.dropna()
        if len(s)>0:
            ts.loc[t] = month_sum(s)
        else:
            ts.loc[t] = np.nan
    return ts

for i in range(len(table1)):
    a = mean_sum(table1.index[i])
#    a = a.sort_index().iloc[31:,:] # 从2000.1开始
    table1.iloc[i,:] = a.mean()

In [3]:
table1 = pd.read_csv('table1.csv',index_col = 0)
table1.round(2)

,mean,SD,skew,kurt,min,5%,25%,median,75%,95%,max,n
illiq1M,0.24,1.22,16.81,527.86,0.00,0.02,0.07,0.14,0.26,0.53,42.69,1705.93
illiq3M,0.22,0.61,13.14,353.61,0.00,0.03,0.08,0.15,0.26,0.53,17.47,1653.55
illiq6M,0.23,0.63,12.57,334.61,0.00,0.03,0.09,0.16,0.27,0.53,18.91,1611.18
illiq12M,0.23,0.52,11.91,309.21,0.00,0.03,0.09,0.17,0.27,0.53,14.79,1558.98
lnilliq1M,-2.73,0.97,-0.26,2.65,-6.68,-4.44,-3.29,-2.63,-2.08,-1.40,1.93,1705.93
lnilliq3M,-2.63,0.93,-0.37,2.19,-6.51,-4.29,-3.16,-2.53,-2.00,-1.35,1.47,1653.55
lnilliq6M,-2.56,0.92,-0.38,2.11,-6.42,-4.20,-3.08,-2.46,-1.95,-1.31,1.45,1611.18
lnilliq12M,-2.47,0.91,-0.41,1.98,-6.29,-4.08,-2.97,-2.38,-1.88,-1.25,1.39,1558.98


![Markdown](http://i1.fuimg.com/716668/ef23b5411da87635.png)

## 13.3 Correlations

In [ ]:
beta = pd.read_csv('beta.csv',index_col = 0).iloc[:,25:]
beta.columns = ts.index
size = pd.read_csv('mktcap.csv')
size['size'] = np.log(size.mktcap)
size = size[(size.type == 1) | (size.type == 4)]
size.date = pd.to_datetime(size.date)
size = size.pivot(index='code', columns='date', values='size')
size = size.iloc[:,36:]
bm = pd.read_csv('bm.csv',index_col = 0).iloc[:,-240:]
bm.columns = size.columns 
mom = pd.read_csv('Mom.csv',index_col = 0)
rev = pd.read_csv('rev.csv',index_col = 0)
rev.columns = size.columns
def corr(t):
    bdata = pd.DataFrame([beta.iloc[:,t],size.iloc[:,t],bm.iloc[:,t],
                          mom.iloc[:,t],rev.iloc[:,t]]).T
    bdata.columns = ['beta','size','bm','mom','rev']
    subliqdata = liqdata[liqdata.index == beta.columns[t]]
    subliqdata = subliqdata.set_index('code',drop = True)
    bdata = subliqdata.join(bdata)
    bdata = bdata.dropna()
    if len(bdata)>0:
        a = bdata.corr('pearson')
        b = bdata.corr('spearman')
    else:
        a=b=0
    return a,b
prs,sprm = corr(0)
for t in range(1,240):
    a,b = corr(t)
    prs += a
    sprm += b
    print(t)
prs /= prs.iloc[0,0]
sprm /= prs.iloc[0,0]
table2 = prs
for i in range(5):
    for j in range(5):
        if i == j:
            table2.iloc[i,j] = np.nan
        if i < j:
            table2.iloc[i,j] = sprm.iloc[i,j]

In [4]:
table2 = pd.read_csv('table2.csv',index_col = 0)
table2.round(2)

,illiq1M,illiq3M,illiq6M,illiq12M,lnilliq1M,lnilliq3M,lnilliq6M,lnilliq12M,beta,size,bm,mom,rev
illiq1M,NaN,0.94,0.90,0.85,1.00,0.70,0.67,0.62,-0.08,-0.52,-0.01,-0.14,-0.05
illiq3M,0.87,NaN,0.96,0.90,0.94,0.76,0.73,0.68,-0.09,-0.55,-0.02,-0.15,0.00
illiq6M,0.79,0.91,NaN,0.95,0.90,0.73,0.76,0.72,-0.09,-0.56,-0.02,-0.13,0.01
illiq12M,0.65,0.74,0.80,NaN,0.85,0.62,0.65,0.72,-0.10,-0.51,-0.02,-0.04,0.01
lnilliq1M,0.75,0.71,0.68,0.58,NaN,0.95,0.91,0.87,0.04,-0.80,-0.03,-0.24,-0.08
lnilliq3M,0.70,0.76,0.73,0.62,0.95,1.00,0.97,0.92,0.03,-0.82,-0.03,-0.24,0.00
lnilliq6M,0.67,0.73,0.76,0.65,0.91,0.97,1.00,0.96,0.03,-0.83,-0.04,-0.20,0.01
lnilliq12M,0.62,0.68,0.72,0.72,0.87,0.92,0.96,1.00,0.03,-0.83,-0.06,-0.11,0.02
beta,-0.08,-0.09,-0.09,-0.10,0.04,0.03,0.03,0.03,1.00,-0.20,-0.04,-0.10,-0.03
size,-0.52,-0.55,-0.56,-0.51,-0.80,-0.82,-0.83,-0.83,-0.20,1.00,-0.00,0.21,0.06


- Illiquidity has a strong negative and hightly monotonic relation with stock size.
- Illiquidity exhibits a negative relation with Mom. 

## 13.4 Persistence


In [ ]:
ts = beta.columns 
def pst():
    for lag in [1,3,6,12,24,36,48,60,120]:
        lagts = pd.DataFrame(index = range(lag,240),columns = liqdata.columns[1:])
        for i in range(lag,len(ts)):
            pdata = liqdata[liqdata.index == ts[i]]
            pdata = pdata.set_index('code',drop = True)
            lagdata = liqdata[liqdata.index == ts[i-lag]]
            lagdata = lagdata.set_index('code',drop = True)
            lagdata.columns = range(8)
            pldata = pdata.join(lagdata)
            for k in range(8):
                subpldata = pldata.iloc[:,[k,k+8]]
                subpldata = subpldata.dropna()
                lagts.loc[i][k] = np.corrcoef(subpldata.iloc[:,0],subpldata.iloc[:,1])[0][1]
        table3.loc[lag] = lagts.mean()
        print(lag)
            

table3 = pd.DataFrame(index = [1,3,6,12,24,36,48,60,120],
                      columns = liqdata.columns[1:])
pst()            

In [7]:
table3 = pd.read_csv('table3.csv',index_col = 0)
table3.round(2)

,illiq1M,illiq3M,illiq6M,illiq12M,lnilliq1M,lnilliq3M,lnilliq6M,lnilliq12M
1,0.51,0.71,0.83,0.91,0.87,0.95,0.97,0.99
3,0.41,0.54,0.66,0.73,0.81,0.87,0.93,0.95
6,0.34,0.47,0.54,0.65,0.76,0.82,0.86,0.91
12,0.28,0.38,0.43,0.50,0.70,0.75,0.78,0.82
24,0.25,0.32,0.36,0.40,0.62,0.67,0.70,0.73
36,0.20,0.27,0.30,0.34,0.58,0.62,0.65,0.68
48,0.18,0.26,0.29,0.31,0.54,0.59,0.61,0.64
60,0.16,0.22,0.23,0.24,0.50,0.55,0.57,0.60
120,0.13,0.17,0.19,0.21,0.37,0.40,0.42,0.43


- Illiq is a highly persistent variable
- log-transformed measures are better at capturing a cross-sectionally persistent characteristic 

## 13.5 Liquidity and Stock Returns
- less liquid stocks command higher expected returns while more liquid stocks command lower expected returns.
### Univariate Portfolio Analysis
- it is highly illiquid stocks, specifically those stocks in the highest illiquidity decile, that are driving the positive relation between illiquidity and average returns

In [ ]:
liqdata1['date'] = liqdata1.index
illiq12 = liqdata1.pivot(index='code', columns='date', values='illiq12M')
lnilliq12 = liqdata1.pivot(index='code', columns='date', values='lnilliq12M')

mktcap = pd.read_csv('mktcap.csv')
mktcap = mktcap[(mktcap.type == 1) | (mktcap.type == 4)]
mktcap.date = pd.to_datetime(mktcap.date)
mktcap = mktcap.pivot(index='code', columns='date', values='mktcap')
mktcap = mktcap.iloc[:,36:]
beta = beta.loc[illiq12.index]
mktcap = mktcap.loc[illiq12.index]
bm = bm.loc[illiq12.index]
mom = mom.loc[illiq12.index]
rev = rev.loc[illiq12.index]
beta.columns = mktcap.columns = bm.columns = mom.columns = rev.columns = illiq12.columns
mktcap = mktcap/1000
table4pA = PortChar(illiq12, [illiq12, lnilliq12, beta, mktcap, bm, mom, rev])
table4pA.index = ['illiq12', 'lnilliq12', 'beta', 'mktcap', 'bm', 'mom', 'rev']
table4pA.columns = range(1,11)

In [8]:
table4pA = pd.read_csv('table4pA.csv',index_col = 0)
table4pA.round(2)

,1,2,3,4,5,6,7,8,9,10
illiq12,0.03,0.07,0.09,0.12,0.15,0.19,0.23,0.28,0.35,0.81
lnilliq12,-4.25,-3.37,-2.98,-2.71,-2.48,-2.28,-2.08,-1.88,-1.64,-1.07
beta,1.01,1.09,1.10,1.11,1.11,1.11,1.11,1.10,1.08,1.00
mktcap,44182.26,8877.45,5714.16,4287.15,3446.34,2865.60,2438.14,2076.98,1776.54,1483.75
bm,1.04,0.80,0.80,0.83,0.83,0.85,0.86,0.89,0.88,0.83
mom,24.72,22.91,19.38,16.94,16.99,15.56,14.36,13.78,14.07,16.02
rev,1.04,0.99,1.01,1.14,1.16,1.25,1.46,1.53,1.66,1.85


In [ ]:
fac = pd.read_csv('fivefactor_monthly.csv',index_col = 0)
fac = fac[['rf','mkt_rf','smb','hml','umd']].iloc[71:-3,:]
fac.index = illiq12.columns 
revfac = pd.read_csv('rev_factor.csv',index_col = 0)
revfac.index = fac.index
fac['rev'] = revfac.iloc[:,1]

ret = pd.read_csv('mktcap.csv')
ret = ret[(ret.type==1)|(ret.type==4)].iloc[:,:-1]
Rfree = fac.rf
ret.date = pd.to_datetime(ret.date)
ret.date = ret.date.apply(lambda x: x+ relativedelta(months=1) - relativedelta(days=1))
ret = ret[ret.date > '20000101']
ret = ret.set_index('date',drop=True)
ret.rt = ret.rt - Rfree[ret.index] # 处理得到了超额收益率
ret['date'] = ret.index

liqretdata = liqdata1.merge(ret,on = ['date','code'],how = 'inner')

def wmean(data):
    return np.average(data.rt,weights = data.cap)
def retts11(udata):
    # 得到11个组的超额收益率序列, 返回两个序列，分别是等权 & 市值加权
    # udata: DataFrame: code date ret mktcap 
    udata.columns = ['code','date','rt','illiq','cap']
    liqshift = udata[['code','date','illiq']]
    udata = udata[['code','date','rt','cap']]
    liqshift.date = liqshift.date.apply(lambda x: x+ relativedelta(days=1)+ relativedelta(months=1)-relativedelta(days=1))
    lcdata = udata.merge(liqshift, on =['date','code'],how = 'outer')
    
    retts1 = pd.DataFrame(index = ts[1:],columns=range(1,11))
    retts2 = pd.DataFrame(index = ts[1:],columns=range(1,11))
    
    for t in ts[1:]:
        tdata = lcdata[lcdata.date==t].dropna()
        if len(tdata)>0:
            tdata = tdata.sort_values('illiq').reset_index(drop = True)
            tdata['deci'] = np.trunc(tdata.index/(len(tdata)/10))+1
            retts1.loc[t] = tdata[['deci','illiq','cap','rt']].groupby('deci').mean().rt.T.values.tolist()
            retts2.loc[t] = tdata[['deci','illiq','cap','rt']].groupby('deci').agg(wmean).rt.T.values.tolist()
        else:
            continue
        print(t)
    retts1['10-1'] = retts1.iloc[:,9]-retts1.iloc[:,0]
    retts2['10-1'] = retts2.iloc[:,9]-retts2.iloc[:,0]
    return retts1,retts2

port10ts = []
for var in ['illiq3M','illiq6M','illiq12M']:
    ew,vw = retts11(liqretdata[['code','date','rt',var,'mktcap']])
    port10ts.append(ew)
    port10ts.append(vw)
        
def NWtest_1sample(a, lags=6):
    adj_a = np.array(a)
    # 对常数回归
    model = sm.OLS(adj_a, [1] * len(adj_a)).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return adj_a.mean(), float(model.tvalues)

def portret(retts):
    # retts: DataFrame: 11列收益率
    # mkt: Series: 市场因子
    pBC = pd.DataFrame(index = ['ret','ret_t','alpha','alpha_t'],columns = capts_ew.columns)
    for i in range(11):
        pBC.iloc[0,i],pBC.iloc[1,i] = NWtest_1sample(retts.iloc[:,i].tolist())
        model = sm.OLS(retts.iloc[:,i].tolist(),sm.add_constant(facdata.mkt_rf))
        model = model.fit(cov_type='HAC',cov_kwds={'maxlags':6})
        pBC.iloc[2,i],pBC.iloc[3,i] = model.params[0],model.tvalues[0]
    return pBC
table4pBC = pd.DataFrame(index = ['illiq1M_ew','illiq3M_ew','illiq6M_ew','illiq12M_ew',
                                  'illiq1M_vw','illiq3M_vw','illiq6M_vw','illiq12M_vw'],
                        columns = [1,2,3,4,5,6,7,8,9,10,'10-1','10-1_t','FFCalpha','FFCalpha_t','FFCSalpha','FFCSalpha_t'])
for i in range(4):
    for j in range(2):
        subts = port10ts[2*i+j]
        subts = subts.dropna()
        table4pBC.iloc[j*4+i,0:11] = subts.mean()
        table4pBC.iloc[j*4+i,11] = NWtest_1sample(subts.iloc[:,-1].dropna().tolist())[1]
        subfac = fac.loc[subts.index]
        model1 = sm.OLS(subts.iloc[:,-1].tolist(),sm.add_constant(subfac[['mkt_rf','smb','hml','umd']])).fit(cov_type='HAC',cov_kwds={'maxlags':6})
        model2 = sm.OLS(subts.iloc[:,-1].tolist(),sm.add_constant(subfac[['mkt_rf','smb','hml','umd','rev']])).fit(cov_type='HAC',cov_kwds={'maxlags':6})
        table4pBC.iloc[j*4+i,12:14] = model1.params[0],model1.tvalues[0]
        table4pBC.iloc[j*4+i,14:16] = model2.params[0],model2.tvalues[0]
        
subts = port10ts[1]
subts = subts.dropna()
subfac = fac.loc[subts.index]
table4pD = pd.DataFrame(index = ['alpha','t','mkt','t','smb','t','hml','t','mom','t','rev','t'],
                        columns = [1,2,3,4,5,6,7,8,9,10,'10-1'])
for i in range(11):
    model = sm.OLS(subts.iloc[:,i].tolist(),sm.add_constant(subfac[['mkt_rf','smb','hml','umd','rev']])).fit(cov_type='HAC',cov_kwds={'maxlags':6})
    table4pD.iloc[[0,2,4,6,8,10],i] = model.params.tolist()
    table4pD.iloc[[1,3,5,7,9,11],i] = model.tvalues.tolist()

![Markdown](http://i1.fuimg.com/716668/5ab1d16b97568c3b.png)

![Markdown](http://i1.fuimg.com/716668/bb29140559458ee4.png)
![Markdown](http://i2.tiimg.com/716668/7aeaab754b8a2529.png)

- The relation appears strongest when Illiq1M is used as the sort variable. (While in US it's Illiq12M)

### Bivariate Portfolio Analysis

In [ ]:
#table4pD = pd.read_csv('table4pD.csv',index_col = 0)
#table4pD.round(3)

In [ ]:
Table5VW = BiDep_all(mktcap, illiq12, Ret, f=fac, method='VW', q=(5, 5), cap=mktcap)
Table5EW = BiDep_all(mktcap, illiq12, Ret, f=fac, method='EW', q=(5, 5))
Table5 = Table5EW.append(Table5VW)

![Markdown](http://i2.tiimg.com/716668/8aed77246cde6370.png)

- The effect is significant in small MktCap portfolios. (Same as US)

In [ ]:
Table6 = pd.concat(
    [
        BiDep_ls(i, illiq12, Ret, f=fac, method=j, q=(5, 5), cap=mktcap) for i, j in zip([beta, beta, bm, bm,mom,mom,rev,rev],
                                                                                        ['VW', 'EW']*4)
    ],
    axis=0
)

TABLE 13.6 Bivariate Dependent-Sort Portfolio Analysis

![Markdown](http://i1.fuimg.com/716668/d66e97d030b14131.png)
![Markdown](http://i1.fuimg.com/716668/bda74696c67daabd.png)

- In equal-weighted portfolio analysis all variables fail to explain the relation.

In [ ]:
Table7 = pd.concat(
    [
        BiInd(i, illiq12, Ret, f=fac, method=j, q=(5, 5), cap=mktcap) for i, j in zip([beta, beta, bm, bm,mom,mom,rev,rev],
                                                                                        ['VW', 'EW']*4)
    ],
    axis=0
)

TABLE 13.7 Bivariate Independent-Sort Portfolio Analysis

![Markdown](http://i2.tiimg.com/716668/cc0f10154356fd10.png)
![Markdown](http://i2.tiimg.com/716668/e02fd4fb8f457da2.png)

In [ ]:
Table8 = pd.concat(
    [
        BiDep_ls(illiq12,i, Ret, f=fac, method=j, q=(5, 5), cap=mktcap) for i, j in zip([beta, beta, bm, bm,mom,mom,rev,rev],
                                                                                        ['VW', 'EW']*4)
    ],
    axis=0
)

#### Can Liquidity Explain Other Relations?

TABLE 13.8 Bivariate Dependent-Sort Portfolio Analysis—Control for Illiq12M
![Markdown](http://i1.fuimg.com/716668/85f16db0741b796f.png)
![Markdown](http://i1.fuimg.com/716668/bf477605b6b59c35.png)

- Controling illiq can't explain any of the four other relations. (No relation between mom and return in China)

In [ ]:
Table9 = pd.concat(
    [
        BiInd(illiq12,i, Ret, f=fac, method=j, q=(5, 5), cap=mktcap) for i, j in zip([beta, beta, bm, bm,mom,mom,rev,rev],
                                                                                        ['VW', 'EW']*4)
    ],
    axis=0
)

TABLE 13.9 Bivariate Independent-Sort Portfolio Analysis—Control for Illiq12M
![Markdown](http://i2.tiimg.com/716668/76ec3c99d3896bf7.png)
![Markdown](http://i2.tiimg.com/716668/20dbd0eee0a21adc.png)

### Fama-Macbeth regression

In [ ]:
Spe = [
    [illiq12], [illiq12, beta], [illiq12, size], [illiq12, bm],
    [illiq12,mom], [illiq12,rev]
]
Pos = [[2], [4], [2, 6], [2,8], [2,10], [2]]
Num = [[10], [8], [2, 6], [4,4], [6,2], [8]]

size.columns = illiq12.columns
table10 = pd.DataFrame(index = range(16),columns = range(1,7))
for i in range(6):
    table10.iloc[:,i] = FMRegression(Ret, Spe[i], Pos[i], Num[i], level=0.005)

Speln = [
    [lnilliq12], [lnilliq12, beta], [lnilliq12, size], [lnilliq12, bm],
    [lnilliq12,mom], [lnilliq12,rev]
]
table10ln = pd.DataFrame(index = range(16),columns = range(1,7))
for i in range(6):
    table10ln.iloc[:,i] = FMRegression(Ret, Speln[i], Pos[i], Num[i], level=0.005)

TABLE 13.10 Fama–MacBeth Regression Analysis—Illiq12M and ln Illiq12M
![Markdown](http://i2.tiimg.com/716668/f14c057cafb1977a.png)

TABLE 13.11 Fama–MacBeth Regression Analysis—Illiq and ln Illiq
![Markdown](http://i2.tiimg.com/716668/34b90e176b894386.png)
![Markdown](http://i2.tiimg.com/716668/c381d495bcde195d.png)

### 13.6 Liquidity Factors (Pastor and Stambaugh)

- Step 1       
    -  generate a measure of stock-level liquidity.
        ![Markdown](http://i1.fuimg.com/716668/ba78517b33c6313e.png)
    - capture only the portion of that move that is not related to changes in the fundamental value of the stock. 
    - what PS capture is whether a given stock tends to realize daily return reversals on days subsequent to days with high trading volume. 
    - high values of 𝛾i,m are associated with highly liquid stocks and low values of 𝛾i,m indicate illiquid stocks
    - very noisy and does not exhibit any ability to predict the cross section of future stock returns, or even the cross section of future liquidity. 

- Step 2
    - aggregate liquidity    
    ![Markdown](http://i2.tiimg.com/716668/d55f3e70d0dc3a45.png)

Figure 13.1 Time-Series Plot of (Vm ∕ V𝟏)̂𝛄m.
![Markdown](http://i2.tiimg.com/716668/3d42b18bd2a4f92c.png)
![Markdown](http://i1.fuimg.com/716668/1c7d398ff876048f.png)

- Step 3
    - Liquidity Innovations
    ![Markdown](http://i1.fuimg.com/716668/55fa23608d8e427c.png)
    - Vm is the total marketcapitalization of the Nm stocks included in the calculation of ̂𝛾m calculated as of the end of month m − 1

-  Step 4
    - ![Markdown](http://i2.tiimg.com/716668/8c7c8b9b62385da0.png)

- Step 5
    ![Markdown](http://i1.fuimg.com/716668/6264cffe5c18e5c7.png)

![Markdown](http://i1.fuimg.com/716668/b7399ebb993f65f4.png)

In [ ]:
a = pd.read_csv('liqdata.csv',index_col = 0)
a.columns = ['code','date','vold','ret']
mktret = pd.DataFrame(a.groupby('date').ret.mean())
a = a.set_index('date',drop = True)
mktret.columns = ['mktret']
a = a.join(mktret)
a.vold = a.vold/1000000
a['y'] = a.ret - a.mktret
a['sgvold'] = a.vold * a.y/abs(a.y)
a.index = pd.to_datetime(a.index)
ts = pd.date_range('19991231','20200101',freq = 'm')


gammaim = pd.DataFrame(index = ts[1:],columns = a.code.unique())

for t in range(1,241):
    suba = a[(a.index <= ts[t]) & (a.index > ts[t-1])]
    for i in suba.code.unique():
        subdata = suba[suba.code == i]
        if len(subdata) > 15:
            model = sm.OLS(subdata['y'][1:].tolist(),sm.add_constant(subdata[['ret','sgvold']].iloc[:-1,:])).fit()
            gammaim[i][t-1] = model.params[2]
        else:
            gammaim[i][t-1] = np.nan
    print(t)
gammaim = gammaim.T
gammam = gammaim.mean()

# 计算Vm
mktcap = pd.read_csv('mktcap.csv')
mktcap = mktcap.pivot(index = 'code',columns = 'date',values = 'mktcap')
mktcap = mktcap.iloc[:,-241:-1] # Vm计算实际上滞后了一期
mktcap.columns = gammaim.columns

Vm = pd.Series(index = mktcap.columns)
for i in range(240):
    used = gammaim.iloc[:,i].dropna().index
    Vm[i] = mktcap.iloc[:,i].loc[used].mean()

gammaim_lag = gammaim.shift(periods=-1, axis=1)
deltagm = (gammaim-gammaim_lag).mean() * Vm / Vm[0]

regdata = pd.DataFrame([deltagm[1:],deltagm[:-1],(gammam * Vm/Vm[0])[:-1]]).T
regdata = regdata.dropna()
model = sm.OLS(regdata[0],sm.add_constant(regdata[[1,2]])).fit()
Lm = model.resid/100

ret = pd.read_csv('mktcap.csv')
ret = ret.pivot(index = 'code',columns = 'date',values = 'rt')
ret = ret.iloc[:,-240:] # Vm计算实际上滞后了一期
ret.columns = gammaim.columns

fac = pd.read_csv('fivefactor_monthly.csv').iloc[71:-3,:]
fac.index = gammaim.columns
regdata2 = fac[['rf','mkt_rf','smb','hml']].join(pd.DataFrame(Lm)).dropna()
regdata2.columns = ['rf','mkt_rf','smb','hml','Lm']

betaL = pd.DataFrame(index = gammaim.index, columns = gammaim.columns)

for i in betaL.index:
    regdata3 = pd.DataFrame(ret.loc[i] - fac.rf).join(regdata2[['mkt_rf','smb','hml','Lm']]).dropna()
    for t in range(60,240):
        subdata3 = regdata3[(regdata3.index<gammaim.columns[t])&(regdata3.index>=gammaim.columns[t-60])]
        if len(subdata3 >= 36):
            model = sm.OLS(subdata3[0].tolist(),sm.add_constant(subdata3.iloc[:,1:])).fit()
            betaL.loc[i][t] = model.params[-1]
        else:
            betaL.loc[i][t] = np.nan
    print(i)

betaL = betaL.iloc[:,60:].T * 1e+18
betaL = betaL.shift(1)
betaL.index = pd.to_datetime(betaL.index)
for i in range(1,180):
    if betaL.index[i].month != 1:
        betaL.iloc[i,:] = np.nan
betaL = betaL.fillna(method = 'ffill',limit = 12).iloc[1:,:]
retL = ret.iloc[:,-180:]
retL = retL.loc[betaL.columns]
capL = cap.iloc[:,-180:]
capL = capL.loc[betaL.columns]
PSL1 = []
for i in range(len(betaL)):
    longlist = (betaL.iloc[i,:]>betaL.iloc[i,:].dropna().quantile(0.9))
    longret = np.average(longlist*retL.iloc[:,i].fillna(0),weights = longlist*capL.iloc[:,i].fillna(0))
    shortlist = (betaL.iloc[i,:]<betaL.iloc[i,:].dropna().quantile(0.1))
    shortret = np.average(shortlist*retL.iloc[:,i].fillna(0),weights = shortlist*capL.iloc[:,i].fillna(0))
    PSL1.append(longret-shortret)
    
plot_Cum_Ret(np.array(PSL1),'Cumulative PSL')

Figure 13.3 Cumulative Returns of PSL Portfolio
![Markdown](http://i2.tiimg.com/716668/0818080471f50183.png)
![Markdown](http://i1.fuimg.com/716668/ab3c18f3ffeada1e.png)